In [ ]:
# Import necessary libraries
import sys
import zipfile
import arcpy
import os
import traceback
from os.path import isdir, join, normpath, split

# Print a message to confirm the libraries are imported
print("Libraries imported successfully.")


In [ ]:
# Function to log messages using arcpy
def logMessage(msg):
    arcpy.AddMessage("[py] " + msg)

# Test the logging function
logMessage("Logging function defined.")


In [ ]:
# Function to extract contents of a ZIP file
def unzipfiles(path, zip):
    shapefileName = ''  # Initialize the shapefile name
    if not isdir(path):  # Check if the directory exists
        os.makedirs(path)  # Create it if necessary

    for x in zip.namelist():  # Loop through items in the ZIP
        arcpy.AddMessage("Extracting " + os.path.basename(x) + " ...")

        # Check for shapefile extension
        fileExt = os.path.splitext(x)[1].lower()
        if fileExt == ".shp":
            shapefileName = normpath(str(x))

        # Handle nested directories and write files
        if not x.endswith('/'):
            root, name = split(x)
            directory = normpath(join(path, root))
            if not isdir(directory):
                os.makedirs(directory)
            open(join(directory, name), 'wb').write(zip.read(x))

    return shapefileName  # Return the shapefile name

# Confirm function definition
logMessage("unzipfiles function defined.")


In [ ]:
# Function to handle the unzipping process
def unzip(infile, outfol):
    try:
        zip = zipfile.ZipFile(infile, 'r')  # Open the ZIP file
        shapefileName = unzipfiles(outfol, zip)  # Extract files
        zip.close()
        return shapefileName
    except:
        tb = sys.exc_info()[2]
        tbinfo = traceback.format_tb(tb)[0]
        pymsg = "PYTHON ERRORS:\nTraceback Info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_type) + ": " + str(sys.exc_value) + "\n"
        arcpy.AddError(pymsg)
        return None

# Confirm function definition
logMessage("unzip function defined.")


In [ ]:
# Function to project the shapefile to Web Mercator
def project(shapefilename):
    inShp = os.path.join(arcpy.env.scratchFolder, shapefilename)  # Input shapefile path
    root, name = split(inShp)
    outShp = os.path.join(root, "_" + name)  # Output shapefile path
    spref = arcpy.SpatialReference(102100)  # Web Mercator projection
    arcpy.Project_management(inShp, outShp, spref)  # Perform reprojection
    return outShp

# Confirm function definition
logMessage("project function defined.")


In [ ]:
if __name__ == '__main__':
    logMessage("Script started")

    try:
        # Debugging environment variables
        logMessage("arcpy.env.scratchFolder: " + str(arcpy.env.scratchFolder))
        logMessage("arcpy.env.scratchWorkspace: " + str(arcpy.env.scratchWorkspace))
    except:
        logMessage("No scratch environment variables available.")

    try:
        # Retrieve input parameters
        p5 = arcpy.GetParameter(0)  # Input ZIP file
        logMessage("File: " + str(p5))

        # Check for valid input
        if p5 == '#' or not p5 or str(p5) == '':
            logMessage("No ZIP file provided.")
        else:
            shapefileName = unzip(str(p5), arcpy.env.scratchFolder)  # Unzip the file
            logMessage("Shapefile: " + shapefileName)

            # Project the shapefile if found
            if shapefileName:
                shapefile2 = project(shapefileName)
                arcpy.SetParameterAsText(1, shapefile2)  # Output the projected shapefile
    except:
        # Handle any unexpected errors
        logMessage("Bad parameters")
        logMessage(sys.exc_info()[0])

    logMessage("Script finished.")


How to Use
Run the script chunk by chunk in Jupyter Notebook.
Make sure to replace the placeholder values (like p5) with real input files when running.
Review output from the arcpy.AddMessage calls in ArcGIS or substitute them with print statements for standalone testing.
